In [1]:
# STEP 1: Install required packages
!pip install -q langchain openai chromadb tiktoken unstructured pdf2image pypdf python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -q pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.8 MB/s eta 0:00:00


In [3]:
pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 1.6 MB/s eta 0:00:00


In [4]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.1 MB/s eta 0:00:00


In [5]:
# STEP 2: Import libraries
import os
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI, AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from getpass import getpass

In [6]:
# STEP 3: Set OpenAI API Key
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

Enter your OpenAI API Key: ··········


In [7]:
# STEP 4: Upload your PDF
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving HR Manual V 2.0.5.7_31012025112812.pdf to HR Manual V 2.0.5.7_31012025112812.pdf


In [9]:
!apt-get install -y poppler-utils
!pip install -q "unstructured[local-inference]" "pdf2image" "pillow-heif" "pyheif" pi-heif

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (305 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
# STEP 5: Load PDF and Split into Chunks
loader = UnstructuredPDFLoader(pdf_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

In [11]:
# STEP 6: Create Embeddings and Store in Chroma VectorDB
#embedding_model = OpenAIEmbeddings()
embeddings = AzureOpenAIEmbeddings(azure_deployment="needle-embedding-ada-002",
                                        model="text-embedding-ada-002",
                                        api_key="8193b561cf7f4a0898de3a53fdadda4c",
                                        azure_endpoint="https://prd-genai-openai-core-ws.openai.azure.com/",
                                        openai_api_type="azure",
                                        api_version="2023-08-01-preview",
                                        chunk_size="4000")
vectordb = Chroma.from_documents(docs, embeddings, persist_directory="chromadb")
vectordb.persist()

/tmp/ipython-input-11-2225286399.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [12]:
# STEP 7: Build RetrievalQA Chain
retriever = vectordb.as_retriever()
# model = AzureChatOpenAI(azure_deployment="Needle-Multimodal",
#                               model="gpt-4o",
#                               temperature=0,
#                               api_key="8193b561cf7f4a0898de3a53fdadda4c",
#                               azure_endpoint="https://prd-genai-openai-core-ws.openai.azure.com/",
#                               tiktoken_model_name= "gpt-4o-mini",
#                               openai_api_type="azure",
#                               api_version="2024-12-01-preview",
#                              )

model = AzureChatOpenAI(
    azure_deployment="Needle-Multimodal",  # Replace with your actual deployment name
    model="gpt-35-turbo",                      # Azure's GPT-3.5 Turbo model name
    temperature=0,
    api_key="8193b561cf7f4a0898de3a53fdadda4c",
    azure_endpoint="https://prd-genai-openai-core-ws.openai.azure.com/",
    tiktoken_model_name="gpt-3.5-turbo",       # Tokenizer model for GPT-3.5
    openai_api_type="azure",
    api_version="2024-12-01-preview"           # Confirm this version supports GPT-3.5 on your Azure account
)


qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    return_source_documents=True
)

In [13]:
# STEP 8: Ask Questions!
while True:
    query = input("\nAsk a question (or type 'exit'): ")
    if query.lower() == "exit":
        break
    result = qa_chain({"query": query})
    print("\nAnswer:\n", result["result"])


Ask a question (or type 'exit'): who is viral


/tmp/ipython-input-13-574569434.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})



Answer:
 Viral Hirpara is the President of Priya Softweb Solutions Pvt Ltd.

Ask a question (or type 'exit'): exit
